# 公開データを整理する
公開するデータを整理するタスクです。<br>
公開したいデータを選択し、整理を行って下さい。


## 公開データを整理する
以下の手順に従って公開対象となるデータの整理を行ってください。

### 1. リサーチフローデータをコピーする
親サブフローになっている査読サブフローから論文・論拠データをこのサブフローにコピーして下さい。<br>
査読サブフローを実行していない場合は論文執筆サブフローからコピーしてください。<br>
「リサーチフローのデータをコピーする」セルを実行することで親サブフローになっている論文執筆サブフローのフォルダと論文データのコピーを保存するためのフォルダを表示することができます。<br>
以下の図の説明に従ってデータのコピーを行ってください。<br>
<br>
![データをダウンロードする](./image/RF007006_download.png)<br>
![データをアップロードする](./image/RF007006_upload.png)<br>
親サブフローを変更する場合はメインメニューに戻り親子関係を変更した後、このタスクを実行してください。<br>
論文執筆、査読時に使用しなかったデータの中に公開したいデータがある場合はこのタスクで追加してください。

In [ ]:
# リサーチフローのデータをコピーする
import os

from IPython.core.display import Javascript
import panel as pn

from library.task_director import TaskDirector
from library.utils.access import open_data_folder
from library.utils.config import path_config, message as msg_config
from library.utils.error import NotFoundSubflowDataError
from library.utils.setting.research_flow_status import get_subflow_type_and_id, ResearchFlowStatusOperater
from library.utils.widgets import MessageBox


notebook_name = 'organize_public_data.ipynb'

class Display(TaskDirector):
    """実験サブフローと論拠データのフォルダを表示するクラスです。

    Attributes:
        instance:
            working_path(str): 実行Notebookファイルパス
            _msg_output(MessageBox): メッセージ出力用のボックス
            _form_section(pn.WidgetBox): ボタン等の出力を格納するためのボックス

    """
    def __init__(self, working_path: str) -> None:
        """Displyクラスのコンストラクタです。

        Args:
            working_path (str): 実行Notebookファイルパス

        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

        abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        research_flow_status_file_path = path_config.get_research_flow_status_file_path(abs_root)
        self.reserch_flow_status_operater = ResearchFlowStatusOperater(research_flow_status_file_path)

        # メッセージボックスの設定
        self._msg_output = MessageBox()
        self._msg_output.width = 900
        # 出力用フォームの設定
        self._form_section = pn.WidgetBox()
        # フェーズごとにサブフローを管理する辞書
        self.experiment_subflow = {}
        self.writing_subflow = {}
        self.review_subflow = {}

    @TaskDirector.task_cell("1")
    def generate_folder_buttons(self):
        """データフォルダを表示するためのボタンを生成するメソッドです。"""

        self.doing_task()

        _, subflow_id = get_subflow_type_and_id(self.working_path)
        self.get_parent_info(5, subflow_id)

        # 親サブフローが研究準備サブフローしか存在しない場合はメッセージを格納、表示する
        if not self.experiment_subflow and not self.writing_subflow and not self.review_subflow:
            self._msg_output.update_warning(msg_config.get("organize_argument_data", "not_found_subflow"))
            self._form_section.append(self._msg_output)

        else:
            old_part= os.path.join("publication", subflow_id)

            # 取得した実験サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in self.experiment_subflow.items():
                new_part = os.path.join("experiment", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                self.log.info(new_working_path)
                button = open_data_folder(new_working_path)
                card = pn.Card(
                    button,
                    title = f"{msg_config.get('organize_argument_data', 'title_experiment')}{parent_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

            # 取得した論文執筆サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in self.writing_subflow.items():
                new_part = os.path.join("writing", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                self.log.info(new_working_path)
                button = open_data_folder(new_working_path)
                card = pn.Card(
                    button,
                    title = f"{msg_config.get('organize_argument_data', 'title_writing')}{parent_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

            # 取得した査読サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in self.review_subflow.items():
                new_part = os.path.join("review", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                self.log.info(new_working_path)
                button = open_data_folder(new_working_path)
                card = pn.Card(
                    button,
                    title = f"{msg_config.get('organize_argument_data', 'title_review')}{parent_name}",
                    width = 550,
                    height = 100
                )
                self._form_section.append(card)

        # 公開データを保存するフォルダを開くボタンの作成
        pn.extension(raw_css=[".custom-card { background-color:rgba(255, 255, 0, 0.1);}"])
        button = open_data_folder(self.working_path)
        card = pn.Card(
            button,
            title = f"{msg_config.get('organize_argument_data', 'title_paper_folder')}",
            width = 550,
            height = 100,
            css_classes = ["custom-card"]
        )
        self._form_section.append(card)

        self.done_task()

        pn.extension()
        display(self._form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_parent_info(self, phase_seq_number: int, current_subflow_id: str):
        """親サブフローとなっている実験サブフローを取得するメソッドです。

        Args:
            current_subflow (str): 現在作業を行っているサブフローのID

        Returns:
            dict: 親サブフローとなっている実験サブフローの情報

        """
        # 全ての親サブフローIDを取得
        parent_ids = self.reserch_flow_status_operater.get_parent_ids(phase_seq_number, current_subflow_id)
        # 全ての親サブフロー名を取得

        phase_seq_number -= 1
        for parent_id in parent_ids:
            while phase_seq_number > 1:
                try:
                    parent_name = self.reserch_flow_status_operater.get_flow_name(phase_seq_number, parent_id)
                    if phase_seq_number == 2:
                        self.experiment_subflow[parent_name] = parent_id
                        break

                    elif phase_seq_number == 3:
                        self.writing_subflow[parent_name] = parent_id

                    elif phase_seq_number == 4:
                        self.review_subflow[parent_name] = parent_id

                    self.get_parent_info(phase_seq_number, parent_id)
                    break

                except NotFoundSubflowDataError:
                    phase_seq_number -= 1
        return


Display(os.path.abspath('__file__')).generate_folder_buttons()

### 2. コピーしたデータを整理する
コピーしたデータを公開するのに適したデータとなるように整理してください。<br>
以下のセルを実行して公開データを表示し、公開に適したデータになるように整理を行ってください。

In [ ]:
# 公開データを表示する
import os

import panel as pn
from IPython.core.display import Javascript
from IPython.display import display

from library.utils.access import open_data_folder


folder_name = 'public'
button = open_data_folder(os.path.abspath('__file__'), folder_name)

pn.extension()
display(button)
display(Javascript('IPython.notebook.save_checkpoint();'))

## 公開データにメタデータを付与する
公開データのメタデータ登録を行ってください。<br>
以下のセルを実行してメタデータの登録を行った後、このタスクに戻り作業を再開してください。<br>
GakuNin RDMにメタデータを登録する場合はGakuNin RDM上から手動で行ってください。<br>

In [ ]:
# メタデータを登録するタスクへアクセスするボタンを表示する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button


def access_register_metadata():
    """検証するタスクへアクセスするボタンを表示するメソッドです。"""
    button_width = 500
    url = '../common/metadata.ipynb'
    obj = create_button(
        url=f'{url}?init_nb=true',
        target='_blank',
        msg=msg_config.get('task', 'access_register_metadata_task'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_register_metadata()

## 再現性を確認する
メタデータの検証と再現性の確認を行ってください。<br>
<br>
※ 再現性を確認する機能は現在開発中です。

### メタデータを検証する
論拠データのメタデータの検証を行ってください<br>
セルを実行してメタデータの検証を行った後、このタスクに戻り作業を再開してください。<br>

In [ ]:
# 検証するタスクへアクセスするボタンを表示する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button


def access_validate():
    """検証するタスクへアクセスするボタンを表示するメソッドです。"""
    button_width = 500
    url = '../common/validate.ipynb'
    obj = create_button(
        url=f'{url}?init_nb=true',
        target='_blank',
        msg=msg_config.get('task', 'access_validate_task'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_validate()

## GakuNin RDMに保存する

In [ ]:
# GakuNin RDMに保存する
import os
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import path_config
from library.task_director import TaskDirector
from library.utils.setting import get_data_dir

script_file_name = 'save_data'
notebook_name = script_file_name+'.ipynb'


class DataSaver(TaskDirector):
    """GRDMに保存するクラスです。

    Attributes:
        instance:
            _abs_root_path (str): 絶対パス
            save_form_box(pn.WidgetBox):フォームを格納する。
            save_msg_output(Message):ユーザーに提示するメッセージを格納する。
    """

    def __init__(self, working_path: str) -> None:
        """DataSaver コンストラクタメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

    @TaskDirector.task_cell("1")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        data_dir = get_data_dir(self.working_path)
        abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        source = [os.path.join(data_dir, 'public_data')]
        self.define_save_form(source)
        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))


DataSaver(working_path=os.path.abspath('__file__')).generate_form_section()

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する
import os

from library.task_director import TaskDirector

script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()